# 6.3 语言模型数据集（周杰伦专辑歌词）

In [1]:
import torch
import random
import zipfile

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(torch.__version__)
print(device)

1.4.0
cpu


## 6.3.1 读取数据集

In [4]:
with zipfile.ZipFile('../../data/jaychou_lyrics.txt.zip') as zin:
    with zin.open('jaychou_lyrics.txt') as f:
        corpus_chars = f.read().decode('utf-8')
corpus_chars[:40]

'想要有直升机\n想要和你飞到宇宙去\n想要和你融化在一起\n融化在宇宙里\n我每天每天每'

In [6]:
# 这个数据集有6万多个字符。为了打印方便，我们把换行符替换成空格，然后仅使用前1万个字符来训练模型。
corpus_chars = corpus_chars.replace('\n', ' ').replace('\r', ' ')
corpus_chars = corpus_chars[0:10000]
# print(corpus_chars[:50])

想要有直升机 想要和你飞到宇宙去 想要和你融化在一起 融化在宇宙里 我每天每天每天在想想想想著你 这


## 6.3.2 建立字符索引

In [11]:
# set()创建一个无序不重复元素集
# enumerate(sequence, [start=0])返回一个[(idx，数据)]的枚举对象，start表示下标从0开始

idx_to_char = list(set(corpus_chars)) # 去掉重复的字符，形成一个char的list
char_to_idx = dict([(char, i) for i, char in enumerate(idx_to_char)]) # (char,i)的字典
vocab_size = len(char_to_idx) 
vocab_size


1027

In [9]:
# join() 方法用于将序列中的元素以指定的字符连接生成一个新的字符串

corpus_indices = [char_to_idx[char] for char in corpus_chars] # 每个char对应的index
sample = corpus_indices[:20] # 前20个index
print('chars:', ''.join([idx_to_char[idx] for idx in sample])) # 前二十个index对应的char，逗号隔开
print('indices:', sample) # 前二十个index

chars: 想要有直升机 想要和你飞到宇宙去 想要和
indices: [561, 402, 135, 805, 58, 341, 824, 561, 402, 910, 118, 47, 278, 874, 69, 571, 824, 561, 402, 910]


将以上代码封装在d2lzh_pytorch包里的load_data_jay_lyrics函数中，以方便后面章节调用。调用该函数后会依次得到corpus_indices、char_to_idx、idx_to_char和vocab_size

## 6.3.3 时序数据的采样
### 6.3.3.1 随机采样

In [12]:
# 本函数已保存在d2lzh_pytorch包中方便以后使用
import torch
import random
def data_iter_random(corpus_indices, batch_size, num_steps, device=None):
    # 减1是因为对于长度为n的序列，X最多只有包含其中的前n - 1个字符
    num_examples = (len(corpus_indices) - 1) // num_steps  # 下取整，得到不重叠情况下的样本个数
    example_indices = [i * num_steps for i in range(num_examples)]  # 每个样本的第一个字符在corpus_indices中的下标
    random.shuffle(example_indices) # 打乱序列

    def _data(i):
        # 返回从i开始的长为num_steps的序列
        return corpus_indices[i: i + num_steps]
    if device is None:
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    
    for i in range(0, num_examples, batch_size):
        # 每次选出batch_size个随机样本
        # 此循环有修改
        batch_indices = example_indices[i: i + batch_size]  # 当前batch的各个样本的首字符的下标
        X = [_data(j) for j in batch_indices]
        Y = [_data(j + 1) for j in batch_indices]
        yield torch.tensor(X, device=device), torch.tensor(Y, device=device) # 输出X Y

In [13]:
# 随机取样 两个样本不一定相邻
my_seq = list(range(30))
for X, Y in data_iter_random(my_seq, batch_size=2, num_steps=6):
    print('X: ', X, '\nY:', Y, '\n')

X:  tensor([[18, 19, 20, 21, 22, 23],
        [12, 13, 14, 15, 16, 17]]) 
Y: tensor([[19, 20, 21, 22, 23, 24],
        [13, 14, 15, 16, 17, 18]]) 

X:  tensor([[ 0,  1,  2,  3,  4,  5],
        [ 6,  7,  8,  9, 10, 11]]) 
Y: tensor([[ 1,  2,  3,  4,  5,  6],
        [ 7,  8,  9, 10, 11, 12]]) 



### 6.3.3.2 相邻采样

In [22]:
# 本函数已保存在d2lzh_pytorch包中方便以后使用
def data_iter_consecutive(corpus_indices, batch_size, num_steps, device=None):
    if device is None:
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    # 30/2*2=30 
    corpus_len = len(corpus_indices) // batch_size * batch_size  # 保留下来的序列的长度,截断保证能被batchsize整除
    
    corpus_indices = corpus_indices[: corpus_len]  # 仅保留前corpus_len个字符
    indices = torch.tensor(corpus_indices, device=device)
    # 30x1 --> 2x15 
    indices = indices.view(batch_size, -1)  # resize成(batch_size, *)
    # 15截断 14//6 = 2 只能形成2个batch
    batch_num = (indices.shape[1] - 1) // num_steps # 扣掉每一行后面的一个，保证x=y-1，计算得出batch的数目
    
    # [  [1 2 3] [1 2 3] [1 2 3] 1]变成
    # [1 1 1 ]
    # [2 2 2 ]
    # [3 3 3 ]
    for i in range(batch_num):
        i = i * num_steps
        X = indices[:, i: i + num_steps]
        Y = indices[:, i + 1: i + num_steps + 1]
        yield X, Y

In [25]:
for X, Y in data_iter_consecutive(my_seq, batch_size=2, num_steps=6):
    print('X: ', X, '\nY:', Y, '\n')

X:  tensor([[ 0,  1,  2,  3,  4,  5],
        [15, 16, 17, 18, 19, 20]]) 
Y: tensor([[ 1,  2,  3,  4,  5,  6],
        [16, 17, 18, 19, 20, 21]]) 

X:  tensor([[ 6,  7,  8,  9, 10, 11],
        [21, 22, 23, 24, 25, 26]]) 
Y: tensor([[ 7,  8,  9, 10, 11, 12],
        [22, 23, 24, 25, 26, 27]]) 

